In [1]:
import pyvisa
from sentio_prober_control.Communication.CommunicatorGpib import *
from sentio_prober_control.Sentio.ProberSentio import *
from pymeasure.instruments.keithley import Keithley2450
import numpy as np
import pandas as pd
from time import sleep

Initializing package sentio_prober_control
Initializing package sentio_prober_control.Communication
Initializing package sentio_prober_control.Sentio
Initializing package sentio_prober_control.Sentio.CommandGroups


# Test Auto-Steeping without Contacting and Separation

In [50]:
class CustomCommunicatorGpib(CommunicatorGpib):
   def __init__(self, vendor, address):
       super().__init__(vendor)
       self.address = address
       self.rm = pyvisa.ResourceManager()
       self.instrument = self.rm.open_resource(self.address)
   def set_timeout(self, timeout_ms):
       self.instrument.timeout = timeout_ms
   def send(self, command):
       self.instrument.write(command)
   def read_line(self):
       return self.instrument.read()

In [51]:

# Create the custom communicator with National Instruments card
comm = CustomCommunicatorGpib(GpibCardVendor.NationalInstruments, "GPIB0::13::INSTR")
# Set the timeout period (example: 10000 ms = 10 seconds)
comm.set_timeout(10000) # Timeout is set in milliseconds
# Initialize SentioProber with the communicator
prober = SentioProber(comm)
# Send an identification query to the instrument
comm.send("*IDN?")
response = comm.read_line()
print("Instrument ID: {0}".format(response))

Instrument ID: MPI Corporation,Sentio - Probe System Software Suite,,3.6.7.1093,12/29/2022



In [52]:

landing_count = 1

for l in range(1,landing_count+1):

    print(f'{l} times landing Started')

    col, row, site = prober.map.step_first_die()
    bin_value = 0
    print("Moved to the first die")
    sleep(0.3)
    #prober.vis.snap_image("/Snapshots")
    #print("Screenshot taken")                      
    sleep(1)
    
    
    """ prober.map.step_die(2,1,2)
    bin_value = 0 """
    while True:
        col, row, site = prober.map.subsites.bin_step_next(bin_value)
        print(f'Position {col}, {row} (Site: {site})')
        sleep(0.3)
        #prober.vis.snap_image("/Snapshots")
        #print("Screenshot taken")     
        sleep(1)
        bin_value = 0 if site == 0 else bin_value + 1
        # Run multiple time IV-Sweep if needed. If not, set k = 1
        print("Continuing stepping")
    
    

1 times landing Started
Moved to the first die
Position -1, -1 (Site: 2)
Continuing stepping
Position -1, -1 (Site: 3)
Continuing stepping
Position -1, -1 (Site: 4)
Continuing stepping
Position -1, -1 (Site: 5)
Continuing stepping
Position -1, -1 (Site: 6)
Continuing stepping
Position -1, -1 (Site: 7)
Continuing stepping
Position -1, -1 (Site: 8)
Continuing stepping
Position -1, -1 (Site: 9)
Continuing stepping
Position -1, -1 (Site: 10)
Continuing stepping
Position 0, -1 (Site: 1)
Continuing stepping
Position 0, -1 (Site: 2)
Continuing stepping
Position 0, -1 (Site: 3)
Continuing stepping
Position 0, -1 (Site: 4)
Continuing stepping
Position 0, -1 (Site: 5)
Continuing stepping
Position 0, -1 (Site: 6)
Continuing stepping
Position 0, -1 (Site: 7)
Continuing stepping
Position 0, -1 (Site: 8)
Continuing stepping
Position 0, -1 (Site: 9)
Continuing stepping
Position 0, -1 (Site: 10)
Continuing stepping
Position 1, -1 (Site: 1)
Continuing stepping
Position 1, -1 (Site: 2)
Continuing steppi

ProberException: End of route

# Test chunk for K2182A

In [3]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())

('TCPIP0::KEYSIGH-7F5PUI6::hislip0::INSTR', 'TCPIP0::KEYSIGH-7F5PUI6::inst0::INSTR', 'ASRL1::INSTR', 'GPIB0::6::INSTR')


In [6]:
k2182 = rm.open_resource("GPIB0::6::INSTR")

k2182.write("*RST")

6

# Test MPI Camera Features

In [ ]:
# If successful, add into loop of measurement
prober.vis.snap_image("/Snapshots")

In [ ]:
prober.vis.auto_focus()

# Test for 2182a and 2450

In [2]:
rm = pyvisa.ResourceManager()
print(rm.list_resources())

('USB0::0x05E6::0x2450::04459122::INSTR', 'TCPIP0::KEYSIGH-7F5PUI6::hislip0::INSTR', 'TCPIP0::KEYSIGH-7F5PUI6::inst0::INSTR', 'ASRL1::INSTR', 'GPIB0::6::INSTR', 'GPIB0::13::INSTR', 'GPIB0::18::INSTR')


In [6]:
SMU = rm.open_resource('GPIB0::18::INSTR')
nVolt = rm.open_resource('GPIB0::6::INSTR')

SMU.write('*IDN?')
#nVolt.write('*IDN?')

7

In [14]:
# Configure the SMU as a current source
SMU.write('*RST')  # Reset the instrument
SMU.write('SOUR:FUNC CURR')  # Set the source function to current
SMU.write('SOUR:CURR:RANG 20E-6')  # Set the current range to 20 µA
SMU.write('SENS:FUNC "VOLT"')  # Set the sense function to voltage
SMU.write('SENS:VOLT:RANG 10E-3')  # Set the voltage range to 10 mV
SMU.write('SENS:VOLT:NPLC 1')

SMU.write('OUTP ON')  # Enable the outputv

9

In [9]:
# Configure the nanovoltmeter
nVolt.write('*RST')  # Reset the instrument
nVolt.write('CONF:VOLT:DC 10E-3')  # Set measurement function to DC voltage, 10 mV range

20

In [11]:
# Delta mode parameters
current_positive = 20e-6  # Positive current: 100 µA
current_negative = -20e-6  # Negative current: -100 µA
num_cycles = 10  # Number of cycles to alternate between currents

In [15]:
# Perform the delta mode measurement
data = []

for cycle in range(num_cycles):
    # Apply positive current
    SMU.write(f'SOUR:CURR {current_positive}')  # Set the positive current
    sleep(0.1)  # Wait for settling time
    voltage_positive = float(nVolt.query('READ?'))  # Measure voltage for positive current

    # Apply negative current
    SMU.write(f'SOUR:CURR {current_negative}')  # Set the negative current
    sleep(0.1)  # Wait for settling time
    voltage_negative = float(nVolt.query('READ?'))  # Measure voltage for negative current

    # Compute average voltage for the cycle
    average_voltage = (voltage_positive - voltage_negative) / 2

    # Store positive, negative, and average voltages in the data
    data.append((current_positive, voltage_positive, current_negative, voltage_negative, average_voltage))


# Turn off the SMU output after measurement
SMU.write('OUTP OFF')

# Save data to a CSV file
import csv

with open('delta_mode_data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    # Write headers
    writer.writerow(['Current_Positive (A)', 'Voltage_Positive (V)', 'Current_Negative (A)', 'Voltage_Negative (V)', 'Average Voltage (V)'])
    # Write data
    writer.writerows(data)

In [ ]:
data = []

                    for cycle in range(num_cycles):
  
                        SMU.write(f'SOUR:CURR {current_positive}')  # Set the positive current
                        sleep(0.1)  # Wait for settling time
                        voltage_positive = float(nVolt.query('READ?'))  # Measure voltage for positive current

   
                        SMU.write(f'SOUR:CURR {current_negative}')  # Set the negative current
                        sleep(0.1)  # Wait for settling time
                        voltage_negative = float(nVolt.query('READ?'))  # Measure voltage for negative current

  
                        average_voltage = (voltage_positive - voltage_negative) / 2

    
                        data.append((current_positive, voltage_positive, current_negative, voltage_negative, average_voltage))
                        df = pd.DataFrame(data, columns=['Current_Positive (A)', 'Voltage_Positive (V)', 'Current_Negative (A)', 'Voltage_Negative (V)', 'Average Voltage (V)'])
                        filename = f'Delta_Voltage_{col}_{row}_{site}_{l}.csv'
                        df.to_csv(filename, index=False)

In [21]:
sweep_currents = []
for i in range(100, 0, -5):
           sweep_currents.extend([i, -i])
sweep_currents.append(0)  # Add the 0 current at the end
sweep_currents = [current * 1e6 for current in sweep_currents]
print(sweep_currents)

[100000000.0, -100000000.0, 95000000.0, -95000000.0, 90000000.0, -90000000.0, 85000000.0, -85000000.0, 80000000.0, -80000000.0, 75000000.0, -75000000.0, 70000000.0, -70000000.0, 65000000.0, -65000000.0, 60000000.0, -60000000.0, 55000000.0, -55000000.0, 50000000.0, -50000000.0, 45000000.0, -45000000.0, 40000000.0, -40000000.0, 35000000.0, -35000000.0, 30000000.0, -30000000.0, 25000000.0, -25000000.0, 20000000.0, -20000000.0, 15000000.0, -15000000.0, 10000000.0, -10000000.0, 5000000.0, -5000000.0, 0.0]
